In [1]:
import pandas as pd
import jieba as jb
import re
import numpy as np

0.0

In [2]:
stop_words=[]
f=open('stop_words.txt','r')
for li in f.readlines():
    li=li.replace('\n','')
    stop_words.append(li)

In [3]:
clear_data=pd.read_csv('clear_data.csv')

In [4]:
clear_data.head()

,Unnamed: 0,Unnamed: 0.1,comment,create_time,like,locate,repost,title,tools,like_nor,repost_nor,comment_nor,fuhao,huati1,huati2,biaoqing
0,0,11800,3939,2015-08-17 20:41,3197,无,7019,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.014482,0.059186,0.040136,.... .... ....,NaN,NaN,NaN
1,1,11801,511,2015-08-17 21:38,747,无,1097,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.003384,0.009250,0.005207,.... .... ....,NaN,NaN,NaN
2,2,11802,542,2015-08-17 22:07,2846,无,1328,话说。最近几年来，各种保守老旧的名字已经不能满足英国人了.... 最近英国公布了一份新生儿姓...,微博 weibo.com,0.012892,0.011198,0.005523,.... ... ... ... .......,NaN,NaN,NaN
3,3,11803,723,2015-08-17 23:25,4774,无,596,话说，有个妹纸最近做了个实验：【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反...,微博 weibo.com,0.021626,0.005026,0.007367,...... ....,【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反差？】,NaN,NaN
4,4,11804,96,2015-08-17 23:59,828,无,244,【体验-伦敦】谁说看电影只能啃爆米花的？号称电影院里伙食最好的Edible Cinema把自...,街边捡来的iPhone 6,0.003751,0.002057,0.000978,NaN,【体验-伦敦】,NaN,NaN


In [28]:
test_dic={}
test_dic.keys().__contains__('a')

False

# 分词&去除停用词
 这里要注意顺序，先进行分词，再进行停用词去除

In [37]:
# 词典
word_dict={}

#分词，并去除停用词
for i,raw_title in enumerate(clear_data.title):
    # 先把title去空格
    raw_title=''.join(raw_title.split())
    # 分词
    title_list=jb.cut(raw_title)
    # 去停用词
    final_list=[]
    for ttl in title_list:
        if ttl not in stop_words:
            final_list.append(ttl)
    
    for final_word in set(final_list):
        if word_dict.keys().__contains__(final_word):
            word_dict[final_word]+=1
        else:
            word_dict[final_word]=1
    
    clear_data.loc[i,'cut_words']=' '.join(final_list)

In [38]:
word_dict['BBC']

99

In [39]:
clear_data.head()

,Unnamed: 0,Unnamed: 0.1,comment,create_time,like,locate,repost,title,tools,like_nor,repost_nor,comment_nor,fuhao,huati1,huati2,biaoqing,cut_words,word_bag
0,0,11800,3939,2015-08-17 20:41,3197,无,7019,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.014482,0.059186,0.040136,.... .... ....,NaN,NaN,NaN,BBC 推送 点 25 分 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 四面佛 BBC...,"{22400: 1, 18848: 1, 25060: 1, 22118: 1, 8585:..."
1,1,11801,511,2015-08-17 21:38,747,无,1097,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.003384,0.009250,0.005207,.... .... ....,NaN,NaN,NaN,BBC 推送 点 25 分 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 四面佛 BBC...,"{22400: 1, 18848: 1, 25060: 1, 22118: 1, 8585:..."
2,2,11802,542,2015-08-17 22:07,2846,无,1328,话说。最近几年来，各种保守老旧的名字已经不能满足英国人了.... 最近英国公布了一份新生儿姓...,微博 weibo.com,0.012892,0.011198,0.005523,.... ... ... ... .......,NaN,NaN,NaN,话 说 保守 老旧 名字 英国人 英国 公布 一份 新生儿 姓名 调查报告 权力 游戏 里 ...,"{34048: 2, 29249: 1, 31682: 1, 16950: 1, 36737..."
3,3,11803,723,2015-08-17 23:25,4774,无,596,话说，有个妹纸最近做了个实验：【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反...,微博 weibo.com,0.021626,0.005026,0.007367,...... ....,【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反差？】,NaN,NaN,话 说 有个 妹纸 做 实验 素颜 妹纸 美图 前 美图 约会 软件 男人 反差 数量 整整...,"{768: 1, 14354: 1, 18947: 1, 20900: 1, 31845: ..."
4,4,11804,96,2015-08-17 23:59,828,无,244,【体验-伦敦】谁说看电影只能啃爆米花的？号称电影院里伙食最好的Edible Cinema把自...,街边捡来的iPhone 6,0.003751,0.002057,0.000978,NaN,【体验-伦敦】,NaN,NaN,体验 伦敦 说 电影 只能 啃 爆米花 号称 电影院 里 伙食 EdibleCinema 自...,"{576: 1, 35520: 1, 15747: 1, 6532: 1, 21637: 1..."


# 拿到list的词典，并转换词袋模型

In [41]:
word_li=list(word_dict.keys())
for i,c_words in enumerate(clear_data.cut_words):
    # 对分好的词进行逐一统计
    word_list=c_words.split(' ')
    temp_dic={}
    for word in word_list:
        # 拿到每个词在字典中的index
        temp_index=word_li.index(word)
        # 统计个数
        if temp_index in temp_dic.keys():
            temp_dic[temp_index]+=1
        else:
            temp_dic[temp_index]=1
    clear_data.loc[i,'word_bag']=str(temp_dic)

In [42]:
clear_data.head()

,Unnamed: 0,Unnamed: 0.1,comment,create_time,like,locate,repost,title,tools,like_nor,repost_nor,comment_nor,fuhao,huati1,huati2,biaoqing,cut_words,word_bag
0,0,11800,3939,2015-08-17 20:41,3197,无,7019,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.014482,0.059186,0.040136,.... .... ....,NaN,NaN,NaN,BBC 推送 点 25 分 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 四面佛 BBC...,"{128: 1, 30817: 1, 27203: 1, 30113: 1, 28457: ..."
1,1,11801,511,2015-08-17 21:38,747,无,1097,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.003384,0.009250,0.005207,.... .... ....,NaN,NaN,NaN,BBC 推送 点 25 分 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 四面佛 BBC...,"{128: 1, 30817: 1, 27203: 1, 30113: 1, 28457: ..."
2,2,11802,542,2015-08-17 22:07,2846,无,1328,话说。最近几年来，各种保守老旧的名字已经不能满足英国人了.... 最近英国公布了一份新生儿姓...,微博 weibo.com,0.012892,0.011198,0.005523,.... ... ... ... .......,NaN,NaN,NaN,话 说 保守 老旧 名字 英国人 英国 公布 一份 新生儿 姓名 调查报告 权力 游戏 里 ...,"{33159: 1, 13320: 1, 35614: 1, 4239: 1, 33812:..."
3,3,11803,723,2015-08-17 23:25,4774,无,596,话说，有个妹纸最近做了个实验：【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反...,微博 weibo.com,0.021626,0.005026,0.007367,...... ....,【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反差？】,NaN,NaN,话 说 有个 妹纸 做 实验 素颜 妹纸 美图 前 美图 约会 软件 男人 反差 数量 整整...,"{7745: 2, 26274: 1, 25957: 1, 1830: 1, 11207: ..."
4,4,11804,96,2015-08-17 23:59,828,无,244,【体验-伦敦】谁说看电影只能啃爆米花的？号称电影院里伙食最好的Edible Cinema把自...,街边捡来的iPhone 6,0.003751,0.002057,0.000978,NaN,【体验-伦敦】,NaN,NaN,体验 伦敦 说 电影 只能 啃 爆米花 号称 电影院 里 伙食 EdibleCinema 自...,"{35841: 1, 12994: 1, 12939: 1, 32964: 1, 901: ..."


# 词袋模型转TF-IDF模型
 TF 计算：
 
     某个词出现的频率/这篇文章所有词的总数
 
 IDF 计算：
 
     log（文档总数/[出现该词的文档数+1]）

In [17]:
test=eval(clear_data.word_bag[0])
all_res=np.sum(list(test.values()))
all_res

17

In [18]:
test

{5878: 2,
 8585: 1,
 11535: 1,
 13749: 1,
 15154: 1,
 15485: 1,
 18848: 1,
 22118: 1,
 22400: 1,
 25007: 1,
 25060: 1,
 28543: 2,
 32279: 1,
 34225: 1,
 36185: 1}

In [25]:
# 计算TF
def cal_tf(word_bag):
    # 传入词袋
    word_tfs={}
    # 拿到每个词的index
    words=word_bag.keys()
    # 计算总共出现词的个数
    total_words=np.sum(list(word_bag.values()))
    # 计算词频
    for word in words:
        cnt=int(word_bag[word])
        temp_tf=1.0*cnt/total_words
        word_tfs[word]=temp_tf
    return word_tfs

In [48]:
# 计算 IDF
def cal_idf(word_bag,word_dict,file_num):
    # 拿到词典
    word_list=list(word_dict.keys())
    # 拿到所有词
    words_ind=word_bag.keys()
    # 结果字典
    res={}
    for word_ind in words_ind:
        # 通过序号拿到真的词
        real_word=word_list[word_ind]
        # 拿到该词在哪些文档中出现过
        fre=word_dict[real_word]
        # 计算IDF
        temp=np.log(file_num/(1+fre))
        res[word_ind]=temp
    return res
    

In [53]:
# 计算总的 TF—IDF
def cal_tf_idf(word_bag,word_dict,file_num):
    tf=cal_tf(word_bag)
    idf=cal_idf(word_bag,word_dict,file_num)
    res={}
    for key in tf.keys():
        res[key]=tf[key]*idf[key]
    return res

In [49]:
test_aaa=cal_tf(test)
test_aaa

{5878: 0.11764705882352941,
 8585: 0.058823529411764705,
 11535: 0.058823529411764705,
 13749: 0.058823529411764705,
 15154: 0.058823529411764705,
 15485: 0.058823529411764705,
 18848: 0.058823529411764705,
 22118: 0.058823529411764705,
 22400: 0.058823529411764705,
 25007: 0.058823529411764705,
 25060: 0.058823529411764705,
 28543: 0.11764705882352941,
 32279: 0.058823529411764705,
 34225: 0.058823529411764705,
 36185: 0.058823529411764705}

In [51]:
test_bbb=cal_idf(test,word_dict,clear_data.shape[0])
test_bbb

{5878: 7.949356201821739,
 8585: 7.949356201821739,
 11535: 8.23703827427352,
 13749: 7.726212650507529,
 15154: 8.642503382381683,
 15485: 6.563061840701848,
 18848: 8.23703827427352,
 22118: 8.642503382381683,
 22400: 8.642503382381683,
 25007: 8.23703827427352,
 25060: 8.642503382381683,
 28543: 8.642503382381683,
 32279: 8.642503382381683,
 34225: 8.642503382381683,
 36185: 8.642503382381683}

In [55]:
test_ccc=cal_tf_idf(test,word_dict,clear_data.shape[0])
test_ccc

{5878: 0.9352183766849105,
 8585: 0.46760918834245524,
 11535: 0.48453166319255997,
 13749: 0.4544830970886782,
 15154: 0.5083825519048049,
 15485: 0.3860624612177558,
 18848: 0.48453166319255997,
 22118: 0.5083825519048049,
 22400: 0.5083825519048049,
 25007: 0.48453166319255997,
 25060: 0.5083825519048049,
 28543: 1.0167651038096097,
 32279: 0.5083825519048049,
 34225: 0.5083825519048049,
 36185: 0.5083825519048049}

In [96]:
# 文档总数
file_num=clear_data.shape[0]
for i,wd_bag in enumerate(clear_data.word_bag):
    # str转dic
    wd_bag_dic=eval(wd_bag)
    # 计算tf_idf
    temp_res=cal_tf_idf(wd_bag_dic,word_dict,file_num)
    temp_res_top_5=sorted(temp_res.items(), key=lambda d:d[1], reverse = True)
    # 保存
    clear_data.loc[i,'tf_idf']=str(temp_res)
    clear_data.loc[i,'tf_idf_top5']=str(temp_res_top_5[:5])

In [97]:
pd.set_option('display.max_colwidth',1000)
clear_data[['word_bag','tf_idf','tf_idf_top5']].head()

,word_bag,tf_idf,tf_idf_top5
0,"{128: 1, 30817: 1, 27203: 1, 30113: 1, 28457: 1, 12810: 1, 27111: 1, 6784: 1, 19698: 2, 12670: 1, 7353: 1, 12762: 1, 27322: 1, 11646: 2, 31061: 1}","{128: 0.2460211157908147, 30817: 0.2564068717953562, 27203: 0.41990741091796524, 30113: 0.2984100403470773, 28457: 0.46760918834245524, 12810: 0.24500979811196924, 27111: 0.4544830970886782, 12762: 0.30733980580990106, 19698: 0.7721249224355116, 11646: 0.5565271031710044, 7353: 0.3113293703004164, 6784: 0.4544830970886782, 27322: 0.34172294343090986, 12670: 0.48453166319255997, 31061: 0.3113293703004164}","[(19698, 0.7721249224355116), (11646, 0.5565271031710044), (12670, 0.48453166319255997), (28457, 0.46760918834245524), (27111, 0.4544830970886782)]"
1,"{128: 1, 30817: 1, 27203: 1, 30113: 1, 28457: 1, 12810: 1, 27111: 1, 6784: 1, 19698: 2, 12670: 1, 7353: 1, 12762: 1, 27322: 1, 11646: 2, 31061: 1}","{128: 0.2460211157908147, 30817: 0.2564068717953562, 27203: 0.41990741091796524, 30113: 0.2984100403470773, 28457: 0.46760918834245524, 12810: 0.24500979811196924, 27111: 0.4544830970886782, 12762: 0.30733980580990106, 19698: 0.7721249224355116, 11646: 0.5565271031710044, 7353: 0.3113293703004164, 6784: 0.4544830970886782, 27322: 0.34172294343090986, 12670: 0.48453166319255997, 31061: 0.3113293703004164}","[(19698, 0.7721249224355116), (11646, 0.5565271031710044), (12670, 0.48453166319255997), (28457, 0.46760918834245524), (27111, 0.4544830970886782)]"
2,"{33159: 1, 13320: 1, 35614: 1, 4239: 1, 33812: 1, 30677: 2, 21017: 1, 12890: 2, 6095: 1, 23644: 1, 7581: 1, 25950: 1, 16738: 1, 13350: 1, 30313: 1, 6698: 1, 8301: 1, 17072: 1, 3912: 1, 36147: 1, 14836: 1, 181: 1, 180: 1, 36474: 1, 15935: 1}","{33159: 0.28615602409287144, 13320: 0.0982793320552526, 4239: 0.12664624075311828, 33812: 0.30507549163975994, 30677: 0.35040595384841017, 25950: 0.22654721252123808, 21017: 0.17446954628360584, 12890: 0.40792660492240995, 6095: 0.19412244447109364, 23644: 0.1767149026471775, 7581: 0.14616958641204178, 35614: 0.09209571408548785, 16738: 0.28615602409287144, 13350: 0.06601347990152631, 30313: 0.2944206000674718, 6698: 0.2944206000674718, 8301: 0.2736940894031969, 17072: 0.2944206000674718, 3912: 0.19871698701442622, 36147: 0.19231540135370728, 14836: 0.095508080507188, 181: 0.20087509472272158, 180: 0.17869596505276958, 36474: 0.32009271786598825, 15935: 0.21515889030835064}","[(12890, 0.40792660492240995), (30677, 0.35040595384841017), (36474, 0.32009271786598825), (33812, 0.30507549163975994), (30313, 0.2944206000674718)]"
3,"{7745: 2, 26274: 1, 25957: 1, 1830: 1, 11207: 1, 13320: 1, 1545: 1, 15466: 1, 33935: 1, 304: 1, 23186: 1, 27028: 2, 3568: 1, 22327: 1, 4239: 1, 33034: 1, 6942: 1, 28528: 1}","{7745: 0.45234662075692117, 26274: 0.3281530920350924, 25957: 0.1594660652636366, 1830: 0.21118313872925432, 11207: 0.23456298319001287, 13320: 0.13267709827459104, 1545: 0.3078798366296842, 15466: 0.2849032201607622, 33935: 0.38631063252537645, 304: 0.1946616426209918, 23186: 0.2967226590639737, 27028: 0.6770701205480093, 3568: 0.331380018091971, 22327: 0.24894708681260191, 4239: 0.17097242501670973, 33034: 0.3425371956576815, 6942: 0.3628104510630897, 28528: 0.19031107377151033}","[(27028, 0.6770701205480093), (7745, 0.45234662075692117), (33935, 0.38631063252537645), (6942, 0.3628104510630897), (33034, 0.3425371956576815)]"
4,"{35841: 1, 12994: 1, 12939: 1, 32964: 1, 901: 1, 23366: 1, 2642: 1, 22088: 1, 20043: 1, 11980: 1, 35394: 1, 17550: 1, 29967: 2, 5906: 1, 23805: 1, 31508: 2, 981: 1, 28434: 1, 18075: 1, 23752: 1, 34721: 1, 16610: 1, 2979: 1, 18276: 1, 17574: 1, 22802: 1, 20379: 1, 22354: 1, 24110: 1, 4683: 3, 8624: 1, 13320: 1, 19378: 1, 22260: 1, 2159: 1, 33620: 1, 14836: 2, 1210: 1, 28861: 1, 5759: 1}","{35841: 0.19205563071959297, 12994: 0.06922356428537534, 20043: 0.14881318296280824, 32964: 0.116846958200798, 901: 0.183045294983856, 23366: 0.07328399206709982, 22088: 0.183045294983856, 12939: 0.1387690691018514, 11980: 0.1249350776941627, 35394: 0.16764202

In [98]:
clear_data.loc[0,['title','tf_idf_top5']]

title                                                                                          BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.... BBC记者表示现场伤亡严重.... 
tf_idf_top5    [(19698, 0.7721249224355116), (11646, 0.5565271031710044), (12670, 0.48453166319255997), (28457, 0.46760918834245524), (27111, 0.4544830970886782)]
Name: 0, dtype: object

In [99]:
temp_tf=eval(clear_data.loc[0,'tf_idf'])
sorted(temp_tf.items(), key=lambda d:d[1], reverse = True)

[(19698, 0.7721249224355116),
 (11646, 0.5565271031710044),
 (12670, 0.48453166319255997),
 (28457, 0.46760918834245524),
 (27111, 0.4544830970886782),
 (6784, 0.4544830970886782),
 (27203, 0.41990741091796524),
 (27322, 0.34172294343090986),
 (7353, 0.3113293703004164),
 (31061, 0.3113293703004164),
 (12762, 0.30733980580990106),
 (30113, 0.2984100403470773),
 (30817, 0.2564068717953562),
 (128, 0.2460211157908147),
 (12810, 0.24500979811196924)]

In [100]:
temp_list=list(word_dict.keys())

In [101]:
temp_list[30113]

'爆炸'

# 字典转向量
呃，之前为了存储方便，使用字典来进行词频和tf-idf的统计，现在要聚类了，只能够使用向量来操作了

In [119]:
file_list=[]
dic_len=len(word_dict)
test_set=set()
for tf_idf in clear_data.tf_idf_top5[:5]:    
    temp_li=eval(tf_idf)
    fi_li=np.zeros(dic_len)
    for tp in temp_li:
        test_set.add(tp[0])
        fi_li[tp[0]]=tp[1]
    file_list.append(list(fi_li))

print(file_list)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [112]:
clear_data[['to_list']].head()

,to_list
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [102]:
clear_data.to_csv('processed_data.csv')